# ImageBasedDataMining.ipynb
‹ ImageBasedDataMining.ipynb › Copyright (C) ‹ 2018 › ‹ Andrew Green - andrew.green-2@manchester.ac.uk › This program is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

This program is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details. You should have received a copy of the GNU General Public License along with this program. If not, see http://www.gnu.org/licenses/.

---

18052018    afg    First version, converted from python script for JEDI MadagaSKA

---

This notebook shows how to do image based data mining against a binary outcome, in this case looking for a relationship between radiation dose and whether a patient needed a feeding tube inserted.



In [ ]:
import os
import time
import os.path
import numpy as np
from tqdm import tqdm
import SimpleITK as sitk
import matplotlib.pyplot as plt

%matplotlib inline

Here we define a class that wraps around the outcome/clinical data (which is in a big .csv file). This allows us to query the data for whatever variables we need to.

This is not efficient, but should only need to be done a few times, so nevermind

In [ ]:
import csv
import re

class HNSCCOutcomes:
    def __init__(self):
        self.csvFileName = "/data/projects/idmrct/hnscc/HNSCC-ClinData_date_offset.csv"


    def findPatient(self, patID):
        with open(self.csvFileName, 'r') as hnData:
            self.reader = csv.DictReader(hnData, fieldnames=["TCIA code","Sex","Age", "Date of Birth", "Diag", "Site", "Histology", "Grade","T","N","M",
                                                        "Stage", "Date of Diagnosis", "Last Contact Date", "Follow up duration (day)", "Follow up duration (year)",
                                                        "Follow up duration (month)", "Date of Death", "Survival  (months)", "Alive or Dead",
                                                        "Cause of Death", "Date of recurrence", "Disease-free interval (months)", "Site of recurrence (Distal/Local/ Locoregional)", "Overall Survival Censor",
                                                        "Disease Specific Survival Censor", "Loco-regional Control Censor", "Oncologic Treatment Summary", "Induction Chemotherapy", "Chemotherapy Regimen",
                                                        "Platinum-based chemotherapy", "Received Concurrent Chemoradiotherapy?", "CCRT Chemotherapy Regimen", "Surgery Summary", "RT Total Dose (Gy)", "Dose/Fraction (Gy/fx)",
                                                        "Number of Fractions", "Unplanned Additional Oncologic Treatment", "Smoking History", "Current Smoker", "Received Feeding Tube (Y/N)", "Type of feeding tube", "Date Feeding tube placed",
                                                        "Date Feeding tube removed", "Feeding tube duration (months)", "Feeding tube note", "Height (cm)", "BW Start tx (kg)"," BW stop treat (kg)", "Height (m)", 
                                                        "BMI start treat (kg/m2)", "BMI stop treat (kg/m2)", "Date Start RT", "Date Stop RT", "Total RT treatment time (days)", "Time between pre and post image (months)", 
                                                        "Time from preRT image to start RT (month)", "Time from RT stop to follow up imaging (months)", "Pre-RT L3 Skeletal Muscle Cross Sectional Area (cm2)", 
                                                        "Post-RT L3 Skeletal Muscle Cross Sectional Area (cm2)", "Pre-RT L3 Adipose Tissue Cross Sectional Area (cm2)", "Post-RT L3 Adipose Tissue Cross Sectional Area (cm2)", 
                                                        "Pre-RT L3 Skeletal Muscle Index (cm2/m2)", "Post-RT L3 Skeletal Muscle Index (cm2/m2)", "Pre-RT L3 Adiposity Index (cm2/m2)", "Post-RT L3 Adiposity Index (cm2/m2)"
                                                        "Pre-RT CT-derived lean body mass (kg)", "Post-RT CT-derived lean body mass (kg)", "Pre-RT CT-derived fat body mass (kg)", "Post-RT CT-derived fat body mass (kg)", 
                                                        "PreRT Skeletal Muscle status", "PostRT Skeletal Muscle status"])
            for row in self.reader:
                if re.search(patID, row["TCIA code"]):
                    return row

        return None

---

One of the probelms with our data is that it is very messy. We need to exclude a lot of patients who have other things that might make them need a feeding tube. In the next cell, we filter the data so we only have patients who got the same number of fractions, and had the same type of treatment (i.e. radiotherapy without surgery first, and similar chemotherapy).

---


In [ ]:
outcomes = HNSCCOutcomes()

"""
Exlude:
- pre-RT surgery
- Weird fractionations (i.e. all pt with 40 fx)
- Induction chemo
- CCRT + immuno.
"""

ptsToUse = []
statuses = []

eligible = 0
events = 0
status = 0

nrrOutputDoses = "/data/projects/idmrct/hnscc/deformedDose"
files = os.listdir(nrrOutputDoses)

for f in files:
    ID = f.split('.')[0]
    summary = outcomes.findPatient(ID)["Oncologic Treatment Summary"]
    pegStatus = outcomes.findPatient(ID)["Type of feeding tube"]
    
    # series of negative filters
    if not summary.startswith("CCRT"): ## Miss any surgery/induction chemo
        continue
    elif re.search("\+", summary):## Miss any immunotherapy
        continue
    elif outcomes.findPatient(ID)["Number of Fractions"] == "40":## Skip weird fractionation
        continue
        
        
    eligible += 1
    if pegStatus == "PEG":
        events += 1
        status = 1
    else:
        status = 0

    ptsToUse.append(ID)
    statuses.append(status)
    print(summary, outcomes.findPatient(ID)["Number of Fractions"], pegStatus)

print(eligible, events)

ptsToUse = np.array(ptsToUse)
statuses = np.array(statuses)

Different fractionations have diferent effects because of the different amount of time normal tissue has to recover. Before we can mine data with different fractionations, we need to standardize somehow. This is usually done with a biologically equivalent dose (BED) calculation, or an equivalent dose @ 2 Gy/fraction (EQD2) calculation. Here we will do a simple BED calculation, whereby we just multiply the real dose by a factor to take account of the differences in fractionation.

Also, whether we want to look at 'acute effects' (i.e. things that happen soon) or 'late effects' (i.e. things that happen after a few weeks/months). the way these differences come into the BED calculation is in an $\alpha/\beta$ ratio. A ratio of $\alpha/\beta$ = 3.0 is usual when considering late effects. I have a hunch that the requirement for a feeding tube is probably an acute effect, in which case an $\alpha/\beta$ ratio of 10 is more appropriate

In [ ]:
earlyAlphaBeta = 10.0
lateAlphaBeta  = 03.0

bedDoses = "/data/projects/idmrct/hnscc/bedCorrectedDoses"

for ID in ptsToUse:
    fractionDose = float(outcomes.findPatient(ID)["Dose/Fraction (Gy/fx)"])
    
    BEDfactor = 1.0 + fractionDose/earlyAlphaBeta ## Note, here we assume that feeding tube insertion is an early effect
    
    thisDose = sitk.ReadImage(os.path.join(nrrOutputDoses, "{0:04d}.nii".format(int(ID))))
    thisDose *= BEDfactor
    sitk.WriteImage(thisDose, os.path.join(bedDoses, "{0:04d}.nii".format(int(ID))))
    
    

Now we have corrected for the fractionation, we are ready to do mine the data. To do this, we define a couple of helper functions.

imagesTTest performs a voxel-wise t-test between the two groups, using [Welford's method](https://www.johndcook.com/blog/standard_deviation/) to calculate the mean and variance on the fly.

The doPermutation function performs a single permutation of the label data, we will use it to assess the statistical significance of what we're looking at.

In [ ]:
def doPermutation(doseData, statuses):
    pstatuses = np.random.permutation(statuses)
    permT = imagesTTest(doseData, pstatuses)
    return np.max(permT)

def imagesTTest(doseData, statuses):#
    imageShape = doseData.shape

    noPEGmean = np.zeros_like(doseData[:,:,:,0])
    PEGmean = np.zeros_like(doseData[:,:,:,0]) + np.finfo(float).eps

    noPEGstd = np.zeros_like(doseData[:,:,:,0])
    PEGstd = np.zeros_like(doseData[:,:,:,0]) + np.finfo(float).eps

    pegCount = 0
    noPegCount = 0
    for n, stat in enumerate(statuses):
        if stat == 1 and pegCount == 0:
            pegCount += 1.0
            PEGmean = doseData[:,:,:,n]
        elif stat == 1:
            pegCount += 1.0
            om = PEGmean#.copy()
            PEGmean = om - (doseData[:,:,:,n] - om)/pegCount
            PEGstd = PEGstd + (doseData[:,:,:,n] - om)*(doseData[:,:,:,n] - PEGmean)
        elif stat == 0 and noPegCount == 0:
            noPegCount += 1.0
            noPEGmean = doseData[:,:,:,n]
        elif stat == 0:
            noPegCount += 1.0
            om = noPEGmean#.copy()
            noPEGmean = om - (doseData[:,:,:,n] - om)/noPegCount
            noPEGstd = noPEGstd + (doseData[:,:,:,n] - om)*(doseData[:,:,:,n] - noPEGmean)
            
    pegTTest = (PEGmean - noPEGmean)/np.sqrt(PEGstd + noPEGstd)

    return pegTTest

Below is the actual data mining. We load all the BED doses and stack them into a single numpy array. We then mask out only the region where we did registration (because we hope the registration is ok there).

Then we do a per-voxel t-test and a permutation test to assess significance. The permutation test will take quite a while!

In [ ]:
"""
This cell generates the indices where we will crop the full resolution array. We can then use a much smaller array
to do the calculation, which makes the permutation testing about 50x faster
"""
masks = "/data/projects/idmrct/hnscc/niftyMasks"
regionMask = sitk.GetArrayFromImage(sitk.ReadImage(os.path.join(masks, "0001.nii")))

idxs0_0 = np.where(regionMask > 0)[0][0]
idxs0_1 = np.where(regionMask > 0)[0][-1]

idxs1_0 = np.where(regionMask > 0)[1][0]
idxs1_1 = np.where(regionMask > 0)[1][-1]

idxs2_0 = np.where(regionMask > 0)[2][0]
idxs2_1 = np.where(regionMask > 0)[2][-1]

regionShape = ((idxs0_1 - idxs0_0),(idxs1_1 - idxs1_0),(idxs2_1 - idxs2_0))


## load all doses
allDoses = os.listdir(bedDoses)

nImages = ptsToUse.shape[0]

probe = sitk.ReadImage(os.path.join(bedDoses, allDoses[0]))

imageShape = [*probe.GetSize()[::-1], nImages]

doseData = np.zeros((*regionShape, nImages))
print(doseData.shape)

for n, img in tqdm(enumerate(ptsToUse)):
    thisDose = sitk.GetArrayFromImage(sitk.ReadImage(os.path.join(nrrOutputDoses, "{0}.nii".format(img) ) ) )
    doseData[:,:,:,n] = thisDose[idxs0_0:idxs0_1, idxs1_0:idxs1_1, idxs2_0:idxs2_1]
#     doseData[:,:,:, n] *= regionMask## could suppress data outside?


## Do a per-voxel t-test
start = time.time()
trueT = imagesTTest(doseData, statuses)
trueMax = np.max(trueT)

trueT_full = np.zeros(imageShape[:-1])
trueT_full[idxs0_0:idxs0_1, idxs1_0:idxs1_1, idxs2_0:idxs2_1] = trueT


outputT = sitk.GetImageFromArray(trueT_full)
outputT.SetSpacing(probe.GetSpacing())
outputT.SetOrigin(probe.GetOrigin())
outputT.SetDirection(probe.GetDirection())
sitk.WriteImage(outputT, "test.nii")


## Now do a permutation test
nPerm = 100
permutationDist = np.zeros((nPerm,))
gtCounts = 1.0

# Slow single threaded works but is slow.
for n in tqdm(range(nPerm)):
    pstatuses = np.random.permutation(statuses)
    permT = imagesTTest(doseData, pstatuses)
    permutationDist[n] = np.max(permT)
    if permutationDist[n] >= trueMax:
        gtCounts += 1.0


plt.figure(1, figsize=(16,9))
plt.hist(permutationDist)
plt.xlabel("Maximum t-value")
plt.ylabel("Counts")
plt.axvline(trueMax)


plt.figure(2, figsize=(16,9))
plt.imshow(trueT_full[50,:,:])
plt.show()

print("Global p: {0}".format(gtCounts/nPerm))
permutationDist = np.sort(permutationDist)
